In [11]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import h5py
import pycbc.conversions as convert
import os
import pandas as pd

In [2]:
import matplotlib as mpl

In [3]:
mpl.rcParams['font.family'] = 'serif'

In [4]:
def mchirp(M,q):
    m1 = convert.mass1_from_mtotal_q(M, q)
    m2 = convert.mass2_from_mtotal_q(M, q)
    return(convert.mchirp_from_mass1_mass2(m1, m2))

In [5]:
base_dir = '/home/divyajyoti/ACADEMIC/Projects/IITM_GW/Eccentric_Population/eccentric_pe/'
run_dir_base = os.path.join(base_dir, 'injections/ICTS_sims/transformed')
plot_dir = os.path.join(base_dir, 'plots')
ecc_spin_run_nums = {'PS': 'run01',
                     'eAS': 'run02',
                     'iPS': 'run03'}

In [6]:
pycbc_to_metadata_dict = {
    'chi_eff': 'chi_eff_ref',
    'chi_p': 'chi_p_ref',
    'distance': 'DL',
    'mass1': 'm1',
    'mass2': 'm2',
    'mchirp': 'm_chirp',
    'mtotal': 'M',
    'polarization': 'pol',
    'eccentricity': 'Eccentricity'
}

keys_not_in_metadata = {
    'coa_phase': 0, 
    'delta_tc': 0,
    'loglikelihood': None,
    'logwt': None
}

In [7]:
with h5py.File('/home/divyajyoti/ACADEMIC/Projects/IITM_GW/Eccentric_Population/eccentric_pe/injections/ICTS_sims/Lev2/EccContPrecDiff003/zero_noise/inj1/run01/posterior.hdf', 'r') as f:
    print(f['samples']['mchirp'][:])
    print(f['samples'].keys())

[34.85056013 35.8615798  33.44073958 ... 34.76891839 34.4056409
 35.95279569]
<KeysViewHDF5 ['chi_eff', 'chi_p', 'coa_phase', 'dec', 'delta_tc', 'distance', 'eta', 'inclination', 'loglikelihood', 'logwt', 'mass1', 'mass2', 'mchirp', 'mtotal', 'polarization', 'q', 'ra', 'spin1_a', 'spin1_azimuthal', 'spin1_polar', 'spin2_a', 'spin2_azimuthal', 'spin2_polar']>


In [8]:
def get_param_inj_val_from_metadata(meta_file, param, round_to=4):
    with open(meta_file, 'r') as f:
        metadata_text = f.read().splitlines()

    while(1):
        try:
            metadata_text.remove('')
        except ValueError:
            break
    
    metadata_text = [line.strip().replace('\\t', '') for line in metadata_text]
    for line in metadata_text:
        if '=' not in line or '===>' in line:
            continue
        param_metadata, val = ''.join(line.split()).split('=')
        return_val = val
        if param == 'spin1z' and param_metadata == 'ChiA_ref':
            return_val = eval(val)[-1]
            break
        elif param == 'spin2z' and param_metadata == 'ChiB_ref':
            return_val = eval(val)[-1]
            break
        elif param_metadata == param:
            break
        elif param in pycbc_to_metadata_dict.keys():
            if param_metadata == pycbc_to_metadata_dict[param]:
                break
    else:
        raise ValueError(f'{param} value not found')
    return(round(float(return_val), round_to))

In [9]:
sims_at_highest_lev = {'names':[], 'meta_files':[], 'levs':[]}
for lev in ['Lev4', 'Lev3', 'Lev2']:
    sim_list = list(os.walk(os.path.join(run_dir_base, lev)))[0][1]
    sim_list.sort()
    for sim in sim_list:
        if sim not in sims_at_highest_lev['names']:
            sims_at_highest_lev['names'].append(sim)
            sims_at_highest_lev['meta_files'].append(os.path.join(run_dir_base, 
                                                                  lev, 
                                                                  sim, 
                                                                  'zero_noise',
                                                                  'inj1',
                                                                  'metadata.txt'))
            sims_at_highest_lev['levs'].append(lev)

In [14]:
inj_vals = {
    'sim_names': [],
    'chi_eff': [],
    'chi_p': [],
    'mass1': [],
    'mass2': [],
    'eccentricity': []
}
for sim, meta_file in zip(sims_at_highest_lev['names'], sims_at_highest_lev['meta_files']):
    inj_vals['sim_names'].append(sim)
    for param in list(inj_vals.keys())[1:]:
        inj_vals[param].append(get_param_inj_val_from_metadata(meta_file, param))
inj_vals_df = pd.DataFrame(inj_vals).set_index('sim_names')

In [15]:
inj_vals_df

,chi_eff,chi_p,mass1,mass2,eccentricity
sim_names,,,,,
EccContPrecDiff001,0.0253,0.3000,27.0000,9.0000,0.04
EccContPrecDiff003,-0.0676,0.2236,79.3333,22.6667,0.10
EccContPrecDiff004,0.3474,0.2952,46.6190,42.3810,0.15
EccContPrecDiff005,0.0046,0.4999,50.5000,50.5000,0.18
EccContPrecDiff006,0.2033,0.4217,69.6000,46.4000,0.21
EccContPrecDiff007,0.0025,0.4123,51.4286,20.5714,0.08
EccContPrecDiff008,0.0013,0.5000,20.0000,20.0000,0.18
EccPrecDiff002,0.1997,0.2005,15.0000,15.0000,0.10
ICTSEccParallel01,0.0005,0.3000,40.5000,13.5000,0.04
